In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
DATA_DIR = "newdata/data_excel_finalmatrix_modified/"
FIRST_DER_DIR = "filter_plots/first_derivative/"
COLORED_DIR = "filter_plots/colored_plots/"
if not os.path.exists(COLORED_DIR):
    os.mkdir(COLORED_DIR)

In [5]:
def readfile(filename):
    return pd.read_excel(filename)

In [6]:
def filter_data(data):
    filtered_data = np.array(data)
    fil_data = []
    for x in filtered_data:
        if x[1] != 0:
            fil_data.append((x[1],x[-1]))
    filtered_data = np.array(fil_data)
    return filtered_data

In [7]:
def conservative_smoothing(data,kernel_len):
    pd = int(kernel_len/2)
    ln = len(data)
    newdata = []
    for i in range(pd):
        newdata.append(data[i])
    for i in range(pd,ln - pd):
        #val = np.sum(data[i-pd:i+pd+1,0]*kernel)
        neighbors = data[i-pd:i,0]
        neighbors = np.concatenate((neighbors,data[i+1:i+1+pd,0]))
        if data[i,0] < np.min(neighbors):
            newdata.append((np.min(neighbors),data[i,1]))
        elif data[i,0] > np.max(neighbors):
            newdata.append((np.max(neighbors),data[i,1]))
        else:
            newdata.append((data[i,0],data[i,1]))
    for i in range(ln-pd,ln) :
        newdata.append(data[i])
    return np.array(newdata)

In [8]:
def apply_filter(data,kernel,kernel_len):
    pd = int(kernel_len/2)
    ln = len(data)
    newdata = []
    for i in range(pd):
        newdata.append(data[i])
    for i in range(pd,ln - pd):
        val = np.sum(data[i-pd:i+pd+1,0]*kernel)
        newdata.append((val,data[i,1]))
    for i in range(ln-pd,ln) :
        newdata.append(data[i])
    return np.array(newdata)

In [9]:
def plot_data(data_blue,data_red,saveplace=None):
    plt.figure(figsize=(20,12))
    plt.scatter(data_blue[:,1],data_blue[:,0],c='b')
    max_y = np.max(data_blue[:,0])
    if data_red.shape[0] > 0:
        plt.scatter(data_red[:,1],data_red[:,0],c='r')
        max_y = max(max_y,np.max(data_red[:,0]))
    plt.ylim(max(60,max_y+10),0)
    if saveplace is not None:
        plt.savefig(saveplace)
        plt.clf()
    else:
        plt.show()

In [10]:
THRESHOLD = 40
gausian_kernel = np.array([5,15,60,15,5])/100

In [11]:
def seperate_data(data,first_der):
    a = []
    b = []
    for i,x in enumerate(data):
        if abs(first_der[i][0]) > THRESHOLD :
            b.append((x[0],x[1]))
        else:
            a.append((x[0],x[1]))
    return np.array(a), np.array(b)

In [12]:
for file in os.listdir(DATA_DIR):
    try:
        data = np.array(filter_data(readfile(DATA_DIR+file)))
        data = conservative_smoothing(data,5)
        data = apply_filter(data,gausian_kernel,5)
        first_file_name = file[:-5]+"_conservative_gaussian_firstder.xlsx"
        first_der = np.array(readfile(FIRST_DER_DIR+first_file_name))
        data_blue,data_red = seperate_data(data,first_der)
        plot_data(data_blue,data_red,COLORED_DIR+file[:-5]+"_conservative_gaussian"+".png")
    except:
        print(file)

Harshit_5.xlsx


/home/ayush/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Niks_5.xlsx
deepesh_31_fm.xlsx
Arpit24.xlsx
